In [ ]:
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

In [ ]:
from fenics import *
from ufl_legacy import nabla_grad
from ufl_legacy import nabla_div
from mshr import *
import numpy as np
import sys


k = Constant(45) # steel conductivity coefficient w/m.k
h = Constant(40) # air convection coefficient w/m^2.k
T_inf = Constant(300)      # kelvin
T_0 = Constant(400)        # Kelvin
q = Constant(100)                    # heat flux w/m^2

# Define boundary condition
tol = 1E-14

mesh = UnitSquareMesh(32,32)
V = FunctionSpace(mesh, 'P', 1)

# plot(mesh)

# # defining boundary
# dirichlet = 'on_boundary && (near(x[0], 0, tol) or near(x[0], 1, tol))'
# neumann = 'on_boundary && (near(x[1], 0, tol))'
# robin = 'on_boundary && (near(x[1], 1, tol))'

# defining boundary markers
boundary_markers = MeshFunction("size_t", mesh, mesh.topology().dim()-1, 0)

class BoundaryX0(SubDomain):
  tol = 1E-14
  def inside(self, x, on_boundary):
    return on_boundary and near(x[0], 0, tol)

class BoundaryX1(SubDomain):
  tol = 1E-14
  def inside(self, x, on_boundary):
    return on_boundary and near(x[0], 1, tol)

class BoundaryY0(SubDomain):
  tol = 1E-14
  def inside(self, x, on_boundary):
    return on_boundary and near(x[1], 0, tol)

class BoundaryY1(SubDomain):
  tol = 1E-14
  def inside(self, x, on_boundary):
    return on_boundary and near(x[1], 1, tol)

bx0 = BoundaryX0()
bx1 = BoundaryX1()
by0 = BoundaryY0()
by1 = BoundaryY1()

bx0.mark(boundary_markers, 1)  # Dirichlet x = 0
bx1.mark(boundary_markers, 2)  # Dirichlet x = 1
by0.mark(boundary_markers, 3)  # Neumann   y = 0
by1.mark(boundary_markers, 4)  # Robin     y = 1


# defining ds and dx of boundary markers
ds = Measure('ds', domain=mesh, subdomain_data=boundary_markers)
dx = Measure('dx', domain=mesh)

# defining boundary conditions
bcs = []
for i in range(1,3):
  bc = DirichletBC(V, T_0, boundary_markers, i)

  bcs.append(bc)

# Define variational problem
u = TrialFunction(V)
v = TestFunction(V)

a = k*dot(grad(u), grad(v))*dx + h*u*v*ds(4)
L = - q*v*ds(3) + h*T_inf*v*ds(4)

# Alternatively
# F = k*dot(grad(u), grad(v))*dx + h*u*v*ds(4) - q*v*ds(3) + h*T_inf*v*ds(4)
# a = rhs(F)
# L = lhs(F)


# Compute solution
u = Function(V)
solve(a == L, u, bcs)

# # Plot solution and mesh
# plot(u, title='Temperature')

In [ ]:
# Plot solution and mesh

import matplotlib.pyplot as plt

p = plot(u, title='Temperature')

plt.colorbar(p)

plt.show()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

output_dir = '/content/drive/MyDrive/FEniCS'

# Save solution to file in VTK format
vtkfile = File(output_dir + '/solution.pvd')
vtkfile << u

# # Create XDMF files for visualization output
# xdmffile_u = XDMFFile(output_dir + '/Temperature.xdmf')
# xdmffile_u.write(u)

